In [1]:
class WhileLoop:
    def __init__(self, condition, content, code_before_while, all_code_after_while):
        self.condition = condition
        self.content = content
        self.code_before_while = code_before_while
        self.all_code_after_while = all_code_after_while

In [2]:
import re

def extract_single_outer_while(source):
    indent_regex = re.compile(r'^\s+', flags=re.MULTILINE)
    try: 
        code_before_while = re.sub(indent_regex, '', source[:source.index('while')].strip())
    except ValueError:
        return WhileLoop(None, None, source, None)

    condition_start = re.search('while\s*\(', source).end()
    condition_end = source.index(')', condition_start)
    condition = source[condition_start:condition_end].strip()
    
    # search for the end } by matching the number of { and } in the content
    content_start = source.index('{', condition_end) + 1
    brace_count = 1
    for i, c in enumerate(source[content_start:]):
        if c == '{':
            brace_count += 1
        elif c == '}':
            brace_count -= 1
            if brace_count == 0:
                content_end = content_start + i
                break

    content = re.sub(indent_regex, '', source[content_start:content_end].strip())
    all_code_after_while = re.sub(indent_regex, '', source[content_end+1:].strip())
    return WhileLoop(condition, content, code_before_while, all_code_after_while)

In [3]:
# test for extract_single_outer_while(source)
with open('source.c', 'r') as f:
    source = f.read()
loop = extract_single_outer_while(source)
print("================================")
print(loop.condition)
print("================================")
print(loop.content)
print("================================")
print(loop.code_before_while)
print("================================")
print(loop.all_code_after_while)

y <= y2
int x = x1;
while (x <= xR)
{
plot = 1; 
x++;
}
while (errR < 0)
{
xR++;
errR = errR + 2 * (y2 - y1);
}
errR = errR - 2 * (x2 - x1);
y++;
int errR = (y2 - y1) - 2 * (x2 - x1);
int xR = x1;
int y = y1;
int counter = 0;
while (counter < 10)
{
counter++;
}
counter *= 2;


In [4]:
def extract_multiple_outer_while(source):
    # (node, depth)
    all_loops = []
    all_loops.append((extract_single_outer_while(source), 0))
    while all_loops[-1][0].all_code_after_while and all_loops[-1][0].all_code_after_while.strip():
        all_loops.append((extract_single_outer_while(all_loops[-1][0].all_code_after_while), all_loops[-1][1]+1))
    return all_loops

In [5]:
# test for extract_multiple_outer_while(source)
with open('source.c', 'r') as f:
    source = f.read()
loops = extract_multiple_outer_while(source)
for loop in loops:
    print(loop[0].code_before_while)
    if (loop[0].condition): 
        print("while (", loop[0].condition, ") {")
        print(loop[0].content)
        print("}")


int errR = (y2 - y1) - 2 * (x2 - x1);
int xR = x1;
int y = y1;
while ( y <= y2 ) {
int x = x1;
while (x <= xR)
{
plot = 1; 
x++;
}
while (errR < 0)
{
xR++;
errR = errR + 2 * (y2 - y1);
}
errR = errR - 2 * (x2 - x1);
y++;
}
int counter = 0;
while ( counter < 10 ) {
counter++;
}
counter *= 2;


In [6]:
class WNode:
    def __init__(self, data, children, condition=False):
        self.data = data
        self.children = children
        self.condition = condition
        self.variables = {}

def build_tree(content):
    if (extract_single_outer_while(content).condition == None):
        return WNode(content, [])
    all_loops = []
    all_loops.append(extract_single_outer_while(content))
    while all_loops[-1].all_code_after_while and all_loops[-1].all_code_after_while.strip():
        all_loops.append(extract_single_outer_while(all_loops[-1].all_code_after_while))
    children = []
    for loop in all_loops:
        if loop.content: 
            children.append(WNode(loop.code_before_while, []))
            children.append(WNode(loop.condition, [build_tree(loop.content)], True))
        else: 
            children.append(WNode(loop.code_before_while, []))
    return WNode('', children)

def print_while_tree(node, indent=-1):
    if node.condition:
        print('  '*indent + 'while (' + node.data + ') {')
    else: 
        for line in node.data.splitlines():
            print('  '*indent + line)
    for child in node.children:
        print_while_tree(child, indent+1) 
    if node.variables: 
        print('  '*indent + 'Variables: ', end='')
        for key in node.variables:
            print(key + ' = ' , node.variables[key], end=' ')
    if node.condition:
        print('  '*indent + '}')

In [7]:
# test for build_tree(source)
with open('source.c', 'r') as f:
    source = f.read()
tree = build_tree(source)

print_while_tree(tree)

int errR = (y2 - y1) - 2 * (x2 - x1);
int xR = x1;
int y = y1;
while (y <= y2) {
    int x = x1;
    while (x <= xR) {
      plot = 1; 
      x++;
    }
    while (errR < 0) {
      xR++;
      errR = errR + 2 * (y2 - y1);
    }
    errR = errR - 2 * (x2 - x1);
    y++;
}
int counter = 0;
while (counter < 10) {
  counter++;
}
counter *= 2;


In [8]:
import re

def extract_variables(code):
    # Extract variables that start with a letter
    variables = re.findall(r'\b[a-zA-Z]\w*\b', code)
    return variables

def extract_variables_from_tree(node):
    variables = []
    for line in node.data.splitlines():
        variables += extract_variables(line)
    for child in node.children:
        variables += extract_variables_from_tree(child)
    # remove duplicates and sort
    variables = list(dict.fromkeys(variables))
    variables.sort()
    return variables

In [9]:
# test for extract_variables_from_tree(tree)
with open('source.c', 'r') as f:
    source = f.read()
tree = build_tree(source)
variables = extract_variables_from_tree(tree)
print(variables)

['counter', 'errR', 'int', 'plot', 'x', 'x1', 'x2', 'xR', 'y', 'y1', 'y2']


In [10]:
def extract_modifications(variable, code): 
    # Extract modifications of the variable
    modifications = re.findall(r'\b' + variable + r'\b\s*=\s*[^;]+', code)
    # keep only the right side of the assignment
    modifications = [re.sub(r'\b' + variable + r'\b\s*=\s*', '', modification) for modification in modifications]
    return modifications

In [11]:
# test for extract_modifications(variable, code)
with open('source.c', 'r') as f:
    source = f.read()
tree = build_tree(source)
variable = 'errR'
node = tree.children[1].children[0].children[3].children[0].data
print(node)
modifications = extract_modifications(variable, node)
print(modifications)

xR++;
errR = errR + 2 * (y2 - y1);
['errR + 2 * (y2 - y1)']


In [59]:
def generate_variable_tree(variable, root, modifications): 
    if root == None or root.children == []:
        return None
    for child in root.children: 
        if variable in extract_variables(child.data): 
            modifications += extract_modifications(variable, child.data)
        if variable in extract_variables_from_tree(child):
            generate_variable_tree(variable, child, modifications) 
            # check if variable is in the first modification using regex
            local_modifications = modifications.copy()
            if re.search(r'\b' + variable + r'\b', local_modifications[0]):
                # append the variable itself as the first modification
                local_modifications.insert(0, variable)
            child.variables[variable] = local_modifications
    return root

In [60]:
# test for generate_variable_trees(variables, tree)
with open('source.c', 'r') as f:
    source = f.read()
tree = build_tree(source).children[1]
# print_while_tree(tree)
variable = 'errR'
tree = generate_variable_tree(variable, tree, [])
# print_while_tree(tree)

variable = 'y'
tree = generate_variable_tree(variable, tree, [])
print_while_tree(tree)

while (y <= y2) {
  int x = x1;
  while (x <= xR) {
    plot = 1; 
    x = x + 1;
  }
  while (errR < 0) {
    xR = xR + 1;
    errR = errR + 2 * (y2 - y1);
    Variables: errR =  ['errR', 'errR + 2 * (y2 - y1)']   Variables: errR =  ['errR', 'errR + 2 * (y2 - y1)']   }
  errR = errR - 2 * (x2 - x1);
  y = y + 1;
  Variables: errR =  ['errR', 'errR + 2 * (y2 - y1)', 'errR - 2 * (x2 - x1)'] y =  ['y', 'y + 1'] Variables: errR =  ['errR', 'errR + 2 * (y2 - y1)', 'errR - 2 * (x2 - x1)'] y =  ['y', 'y + 1'] }


In [61]:
def combine_modifications(variable, modifications): 
    if not modifications: 
        return None
    combined = modifications[0]
    for modification in modifications[1:]:
        # search for the variable in the modification
        if re.search(r'\b' + variable + r'\b', modification):
            # if the variable is in the modification, replace it with the combined modification
            combined = re.sub(r'\b' + variable + r'\b', combined, modification)
        else: 
            # if the variable is not in the modification, append it to the combined modification
            combined += modification
    return combined

In [65]:
# test for combine_modifications(variables, tree)
with open('source.c', 'r') as f:
    source = f.read()
tree = build_tree(source).children[1]
variable = 'errR'
tree = generate_variable_tree(variable, tree, [])
modifications = tree.children[0].children[4].variables[variable]
print(modifications)
combined = combine_modifications(variable, modifications)
print(combined)

variable = 'y'
tree = generate_variable_tree(variable, tree, [])
modifications = tree.children[0].children[4].variables[variable]
print(modifications)
combined = combine_modifications(variable, modifications)
print(combined)

variable = 'x'
tree = generate_variable_tree(variable, tree, [])
modifications = tree.children[0].children[0].variables[variable]
print(modifications)
combined = combine_modifications(variable, modifications)
print(combined)

variable = 'y'
tree = build_tree(source)
tree = generate_variable_tree(variable, tree, [])
modifications = tree.children[1].children[0].children[4].variables[variable]
print(modifications)
combined = combine_modifications(variable, modifications)
print(combined)

['errR', 'errR + 2 * (y2 - y1)', 'errR - 2 * (x2 - x1)']
errR + 2 * (y2 - y1) - 2 * (x2 - x1)
['y', 'y + 1']
y + 1
['x1']
x1
['y1', 'y + 1']
y1 + 1


In [13]:
# chatGPT output for generating state machine
def generate_verilog_code():
    # Define the parameters of the state machine
    num_states = 3
    reset_state = 0
    output_state = 2

    # Generate the Verilog code for the state machine
    code = []
    code.append("module state_machine (")
    code.append("  input wire rst,")
    code.append("  input wire clk,")
    code.append("  input wire start,")
    code.append("  output wire done")
    code.append(");")
    code.append("  reg [{}:0] state;".format(num_states-1))
    code.append("  always @(posedge clk) begin")
    code.append("    if (rst) begin")
    code.append("      state <= {};".format(reset_state))
    code.append("    end else begin")
    code.append("      case (state)")
    for i in range(num_states):
        code.append("        {'{:0{}b}'.format(i, num_states)}: begin")
        if i == reset_state:
            code.append("          if (start) begin")
            code.append("            state <= {};".format((i+1) % num_states))
            code.append("          end")
        else:
            code.append("          state <= {};".format((i+1) % num_states))
        code.append("        end")
    code.append("      endcase")
    code.append("    end")
    code.append("  end")
    code.append("  assign done = (state == {});".format(output_state))
    code.append("endmodule")

    # Return the generated code as a single string
    return "\n".join(code)

# Write the generated code to a file
with open("state_machine.sv", "w") as f:
    f.write(generate_verilog_code())